In [1]:
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

In [2]:
# Shell Plus Model Imports
from django.contrib.admin.models import LogEntry
from django.contrib.auth.models import Group, Permission, User
from django.contrib.contenttypes.models import ContentType
from django.contrib.sessions.models import Session
# Shell Plus Django Imports
from django.core.cache import cache
from django.conf import settings
from django.contrib.auth import get_user_model
from django.db import transaction
from django.db.models import Avg, Case, Count, F, Max, Min, Prefetch, Q, Sum, When, Exists, OuterRef, Subquery
from django.utils import timezone
from django.urls import reverse

Drug Inventory Models
- cmsinv.InventoryItem -> inventory_item
    - id, stock_qty

In [3]:
from cmsinv.models import InventoryItem
from drugdb.models import RegisteredDrug, Company
from inventory.models import Item, ItemType, Category, DeliveryItem
from django.utils import timezone

In [6]:
# Check for matching Item, RegisteredDrug, InventoryItem 
for cmsitem in InventoryItem.objects.all():
    if cmsitem.registration_no:
        print(f"CMSID # {cmsitem.id:<5}, reg:{cmsitem.registration_no}")
        try:
            drug_match = RegisteredDrug.objects.get(reg_no=cmsitem.registration_no)
        except:
            drug_match = None
        if drug_match:
            print(f"--Matching RegDrug: {drug_match.name}")
            # Try get Item via cmsitem.registration_no
            item_data = {
                'name': drug_match.name,
                'reg_no': drug_match.reg_no,
                'cmsid': cmsitem.id,
                'item_type': ItemType.objects.get(name='Drug'),
                'date_created': timezone.now(),
                'last_updated': timezone.now(),
                'updated_by': 'cmsman'
            }
            item_obj, created = Item.objects.update_or_create(
                reg_no=cmsitem.registration_no,
                defaults=item_data
            )
            if created:
                print(f"--Created new item: {item_obj.reg_no:<6}|{item_obj.name}")
            else:
                print(f"--Found item #{item_obj.id:<4}|{item_obj.name}")
            drug_match.itemid == item_obj.id
            drug_match.save()
        else:
            print("--No matching reg_no for cmsInvItem. Try match Item.")
            item_data = {
                'name': cmsitem.product_name,
                'reg_no': cmsitem.registration_no,
                'cmsid': cmsitem.id,
                'item_type': ItemType.objects.get(name='Drug'),
                'date_created': timezone.now(),
                'last_updated': timezone.now(),
                'updated_by': 'cmsman'
            }
            item_obj, created = Item.objects.update_or_create(
                cmsid=cmsitem.id, defaults=item_data
            )
            if created:
                print(f"--Created new item: {item_obj.reg_no:<6}|{item_obj.name}")
            else:
                print(f"--Found item #{item_obj.id:<4}|{item_obj.name}")
        

CMSID # 1    , reg:HK42684
--Matching RegDrug: 0.05% CHLORHEXIDINE & 0.5% CETRIMIDE SOL
--Found item #13  |0.05% CHLORHEXIDINE & 0.5% CETRIMIDE SOL
CMSID # 97   , reg:HK43952
--No matching reg_no for cmsInvItem. Try match Item.
--Found item #108 |ABC PLUS TAB
CMSID # 132  , reg:HK57744
--Matching RegDrug: ACERTIL PLUS TAB 2.5 MG/0.625MG
--Found item #143 |ACERTIL PLUS TAB 2.5 MG/0.625MG
CMSID # 142  , reg:HK50578
--Matching RegDrug: ACETAMOL (B) TAB 250MG
--Found item #152 |ACETAMOL (B) TAB 250MG
CMSID # 143  , reg:HK51663
--Matching RegDrug: ACETAMOL (G) TAB 300MG
--Found item #153 |ACETAMOL (G) TAB 300MG
CMSID # 174  , reg:HK42865
--Matching RegDrug: ACICLOVIR TAB 200MG
--Found item #184 |ACICLOVIR TAB 200MG
CMSID # 175  , reg:HK42867
--Matching RegDrug: ACICLOVIR TAB 400MG
--Found item #185 |ACICLOVIR TAB 400MG
CMSID # 180  , reg:HK48741
--Matching RegDrug: ACILAX CREAM 5%
--Found item #190 |ACILAX CREAM 5%
CMSID # 240  , reg:HK03785
--Matching RegDrug: ACTIFED COMPOUND LINCTUS
--Fo

CMSID # 2909 , reg:HK34352
--No matching reg_no for cmsInvItem. Try match Item.
--Found item #2734|CALCIUM LACTATE TAB 300MG 
CMSID # 2976 , reg:HK53137
--Matching RegDrug: CAMIDEX (NF) CAP
--Found item #2799|CAMIDEX (NF) CAP
CMSID # 3060 , reg:HK05069
--Matching RegDrug: CARBIMAZOLE TAB 5MG (SYNCO)
--Found item #2880|CARBIMAZOLE TAB 5MG (SYNCO)
CMSID # 3176 , reg:HK20158
--Matching RegDrug: CECLOR FOR ORAL SUSP 125MG/5ML
--Found item #2990|CECLOR FOR ORAL SUSP 125MG/5ML
CMSID # 3400 , reg:HK51743
--Matching RegDrug: CETIHIS TAB 10MG
--Found item #3193|CETIHIS TAB 10MG
CMSID # 3616 , reg:HK06462
--Matching RegDrug: CHLORPYRIMINE INJ 10MG/ML
--Found item #3380|CHLORPYRIMINE INJ 10MG/ML
CMSID # 3626 , reg:HK44714
--Matching RegDrug: CHLOZINE TAB 25MG
--Found item #3390|CHLOZINE TAB 25MG
CMSID # 3760 , reg:HK53750
--No matching reg_no for cmsInvItem. Try match Item.
--Found item #3502|CIPROCINE 250MG
CMSID # 3777 , reg:HK50797
--Matching RegDrug: CIPROXYL 500 TAB 500MG
--Found item #3519|

--Matching RegDrug: HUDSON NASAL RELIEF SPRAY 0.05%
--Found item #7847|HUDSON NASAL RELIEF SPRAY 0.05%
CMSID # 8570 , reg:HK06334
--No matching reg_no for cmsInvItem. Try match Item.
--Found item #7918|HYDROCHLOROTHIAZIDE
CMSID # 8571 , reg:HK05176
--Matching RegDrug: HYDROCHLOROTHIAZIDE TAB 50MG (SYNCO)
--Found item #7919|HYDROCHLOROTHIAZIDE TAB 50MG (SYNCO)
CMSID # 8595 , reg:HK08072
--Matching RegDrug: HYDROCORTISONE CREAM 1%
--Created new item: HK08072|HYDROCORTISONE CREAM 1%
CMSID # 8596 , reg:HK09443
--Matching RegDrug: HYDROCORTISONE CREAM 1% (VIDA)
--Found item #7944|HYDROCORTISONE CREAM 1% (VIDA)
CMSID # 8601 , reg:HK21173
--Matching RegDrug: HYDROCORTISONE WITH CLIOQUINOL CREAM
--Found item #7949|HYDROCORTISONE WITH CLIOQUINOL CREAM
CMSID # 8624 , reg:HK55614
--Matching RegDrug: HYOSCINE BUTYLBROMIDE INJ BP 20MG/ML (ROTEXMEDICA)
--Found item #7967|HYOSCINE BUTYLBROMIDE INJ BP 20MG/ML (ROTEXMEDICA)
CMSID # 8629 , reg:HK39990
--Matching RegDrug: HYOSCINE SYRUP 5MG/5ML (VICKMANS

CMSID # 13229, reg:HK25552
--Matching RegDrug: PARCEMOL SYRUP FORTE 250MG/5ML (RED)
--Found item #12302|PARCEMOL SYRUP FORTE 250MG/5ML (RED)
CMSID # 13247, reg:HK47276
--Matching RegDrug: PAROTICIN OTIC DROPS
--Found item #12318|PAROTICIN OTIC DROPS
CMSID # 13371, reg:HK29941
--No matching reg_no for cmsInvItem. Try match Item.
--Found item #12436|SUNPEPCIN TAB 20MG
CMSID # 13461, reg:HK52513
--Matching RegDrug: PHARMANIAGA LORATADINE TAB 10MG
--Found item #12524|PHARMANIAGA LORATADINE TAB 10MG
CMSID # 13516, reg:HK21716
--Matching RegDrug: PHENCOL EXPECTORANT
--Found item #12577|PHENCOL EXPECTORANT
CMSID # 13644, reg:HK42908
--No matching reg_no for cmsInvItem. Try match Item.
--Found item #12691|PIPRINHYDRINATE TAB 3MG (QUALITY)
CMSID # 13653, reg:HK44233
--No matching reg_no for cmsInvItem. Try match Item.
--Found item #12699|PIRITON TAB 4MG
CMSID # 13727, reg:HK39379
--Matching RegDrug: PMS-DOCUSATE SODIUM CAP 100MG
--Found item #12769|PMS-DOCUSATE SODIUM CAP 100MG
CMSID # 13733, r

--Matching RegDrug: BEPROSONE OINT 0.05%
--Found item #18096|BEPROSONE OINT 0.05%
CMSID # 19447, reg:HK25346
--Matching RegDrug: AKAMON TAB 1.5MG
--Found item #18106|AKAMON TAB 1.5MG
CMSID # 19455, reg:HK49176
--Matching RegDrug: SYMBICORT TURBUHALER 160/4.5MCG/DOSE
--Found item #18114|SYMBICORT TURBUHALER 160/4.5MCG/DOSE
CMSID # 19467, reg:HK26101
--Matching RegDrug: CARBIMAZOLE TAB 5MG
--Found item #18126|CARBIMAZOLE TAB 5MG
CMSID # 19479, reg:HK29696
--Matching RegDrug: ZINNAT TAB 250MG
--Found item #18138|ZINNAT TAB 250MG
CMSID # 19480, reg:HK44731
--Matching RegDrug: CELEBREX CAP 200MG
--Found item #18139|CELEBREX CAP 200MG
CMSID # 19485, reg:HK53198
--Matching RegDrug: ZETIZINE ORAL SOLUTION 5MG/5ML
--Found item #18144|ZETIZINE ORAL SOLUTION 5MG/5ML
CMSID # 19488, reg:HK08524
--No matching reg_no for cmsInvItem. Try match Item.
--Found item #18147|CHLORAMPHENICOL EYE DROPS 0.5%
CMSID # 19489, reg:HK14923
--Matching RegDrug: VANMYCETIN OPTICOPS EYE OINT 1%
--Found item #18148|VANM

--Found item #18788|ALLOPURINOL TABLETS 100MG
CMSID # 20174, reg:HK60296
--Matching RegDrug: AMBROX 75 SR CAP 75MG
--Found item #18833|AMBROX 75 SR CAP 75MG
CMSID # 20175, reg:HK59516
--Matching RegDrug: AMBROXOL TAB 30MG
--Created new item: HK59516|AMBROXOL TAB 30MG
CMSID # 20204, reg:HK59056
--Matching RegDrug: AMLOD 5 TAB 5MG
--Found item #18863|AMLOD 5 TAB 5MG
CMSID # 20212, reg:HK60436
--No matching reg_no for cmsInvItem. Try match Item.
--Found item #18871|AMLODIPINE 10MG
CMSID # 20249, reg:HK61679
--Matching RegDrug: AMOROLFINE NAIL LACQUER 5% W/V
--Found item #18908|AMOROLFINE NAIL LACQUER 5% W/V
CMSID # 20312, reg:HK58026
--Matching RegDrug: ANLITIN TAB 1.5MG
--Found item #18971|ANLITIN TAB 1.5MG
CMSID # 20326, reg:HK47196
--Matching RegDrug: ANTIMINE SYRUP 2MG/5ML
--Found item #18985|ANTIMINE SYRUP 2MG/5ML
CMSID # 20327, reg:HK62385
--Matching RegDrug: ANTINO TABLETS
--Found item #18986|ANTINO TABLETS
CMSID # 20339, reg:HK42186
--Matching RegDrug: APO-AMILZIDE TAB 50/5MG
--Fo

--Matching RegDrug: LEVOFLOXACIN SANDOZ TAB 500MG
--Found item #21749|LEVOFLOXACIN SANDOZ TAB 500MG
CMSID # 23105, reg:HK58762
--Matching RegDrug: LEVOTHYROXINE TAB 100MCG
--Found item #21764|LEVOTHYROXINE TAB 100MCG
CMSID # 23106, reg:HK60796
--Matching RegDrug: LEVOTHYROXINE TAB 100MCG
--Created new item: HK60796|LEVOTHYROXINE TAB 100MCG
CMSID # 23134, reg:HK61076
--Matching RegDrug: LIKODIN CAPSULE 500MG
--Found item #21793|LIKODIN CAPSULE 500MG
CMSID # 23181, reg:HK62158
--Matching RegDrug: LISORIL-10 TABLETS 10MG
--Found item #21840|LISORIL-10 TABLETS 10MG
CMSID # 23209, reg:HK49975
--Matching RegDrug: LOMAC-20 CAP 20MG (E.C. GRANULES)
--Found item #21868|LOMAC-20 CAP 20MG (E.C. GRANULES)
CMSID # 23253, reg:HK62485
--Matching RegDrug: LOSANORM-50 TABLETS 50MG
--Found item #21912|LOSANORM-50 TABLETS 50MG
CMSID # 23292, reg:HK54015
--Matching RegDrug: LUNGTEC TAB 10MG
--Found item #21951|LUNGTEC TAB 10MG
CMSID # 23303, reg:HK47445
--Matching RegDrug: MA-AMOXYCILLIN CAP 500MG
--Found

--Created new item: HK01640|PARACETAMOL TAB 500MG
CMSID # 30000, reg:chh00001
--No matching reg_no for cmsInvItem. Try match Item.
--Found item #24900|100 OPTICOPS VANMYCETIN
CMSID # 30001, reg:chh00002
--No matching reg_no for cmsInvItem. Try match Item.
--Found item #24901|BUSCOPAN INJECTION 
CMSID # 30002, reg:chh00003
--No matching reg_no for cmsInvItem. Try match Item.
--Found item #24902|ABC Plus Senior Multivitamin Multimineral Formula
CMSID # 30003, reg:chh00004
--No matching reg_no for cmsInvItem. Try match Item.
--Found item #24903|ACIKLOV CREAM 5% W/W 5GM
CMSID # 30004, reg:chh00005
--No matching reg_no for cmsInvItem. Try match Item.
--Found item #24904|ACOUGH CAPLETS 15MG
CMSID # 30005, reg:chh00006
--No matching reg_no for cmsInvItem. Try match Item.
--Found item #24905|ACT-HIB 0.5ML
CMSID # 30006, reg:chh00007
--No matching reg_no for cmsInvItem. Try match Item.
--Found item #24906|ADACEL POLIO-2NEEDLE SYR 1DOSE 0.5ML (MMRV)
CMSID # 30007, reg:chh00008
--No matching reg_

--Found item #24995|METFORMIN HYDROCHLORIDE TABLETS 250MG
CMSID # 30098, reg:HK64762
--Matching RegDrug: PRALUENT SOLUTION FOR INJECTION IN PREFILLED PEN 75MG/ML
--Found item #25000|PRALUENT SOLUTION FOR INJECTION IN PREFILLED PEN 75MG/ML
CMSID # 30100, reg:HK64417
--Matching RegDrug: ORALITE ORANGE ORAL REHYDRATION SALTS
--Found item #25002|ORALITE ORANGE ORAL REHYDRATION SALTS
CMSID # 30101, reg:HK64821
--Matching RegDrug: DEXAMET SOLUTION FOR INJECTION 5MG/ML
--Found item #25003|DEXAMET SOLUTION FOR INJECTION 5MG/ML
CMSID # 30104, reg:HK64889
--Matching RegDrug: TENOFOVIR DISOPROXIL FUMARATE TABLETS 300MG
--Found item #25006|TENOFOVIR DISOPROXIL FUMARATE TABLETS 300MG
CMSID # 30107, reg:HK64738
--Matching RegDrug: EMANERA GASTRO-RESISTANT CAPSULES 20MG
--Found item #25007|EMANERA GASTRO-RESISTANT CAPSULES 20MG
CMSID # 30105, reg:HK65901
--Matching RegDrug: APTOXIA TABLETS 90MG
--Found item #25009|APTOXIA TABLETS 90MG
CMSID # 30106, reg:HK65900
--Matching RegDrug: APTOXIA TABLETS 60M

In [4]:
cmsitem = InventoryItem.objects.get(id=19963)

In [5]:
cmsitem

<InventoryItem: HK42682 | 0.015% CHLORHEXIDINE & 0.15% CETRIMIDE SOL / CETRIMIDE / CHLORHEXIDINE [0.015% CHLORHEXIDINE & 0.15% ]>

In [6]:
item = Item.objects.filter(reg_no=cmsitem.registration_no)

In [7]:
item

<QuerySet [<Item: 0.015% CHLORHEXIDINE & 0.15% CETRIMIDE SOL>]>

In [8]:
drug_obj = RegisteredDrug.objects.get(reg_no=cmsitem.registration_no)

In [9]:
drug_obj

<RegisteredDrug: HK42682 | 0.015% CHLORHEXIDINE & 0.15% CETRIMIDE SOL>

In [10]:
# Clean Items
used_items = set()
for delivery_item in DeliveryItem.objects.all():
    used_items.add(delivery_item.item.id)
print(f"Items in DeliveryItem: {len(used_items)}")
    

Items in DeliveryItem: 137


In [13]:
matching_cmsitems = set()
for item in Item.objects.all():
    #print(f"Item #{item.id}; CMSid #{item.cmsid}")
    try:
        cmsitem = InventoryItem.objects.get(id=item.cmsid)
    except:
        cmsitem = None
    if cmsitem:
        print(f"Found CMS item with id #{cmsitem.id}")
        matching_cmsitems.add(item.id) 
print(f"Matching cmsItems in Item: {len(matching_cmsitems)}")


Item #18622; CMSid #19963
Found CMS item with id #19963
Item #13; CMSid #1
Found CMS item with id #1
Item #15; CMSid #3
Item #19; CMSid #7
Item #20; CMSid #8
Item #21; CMSid #9
Item #16; CMSid #4
Item #17; CMSid #5
Item #18; CMSid #6
Item #22; CMSid #10
Item #23; CMSid #11
Item #24; CMSid #12
Item #25; CMSid #13
Item #14; CMSid #2
Item #29; CMSid #17
Item #30; CMSid #18
Item #31; CMSid #19
Item #26; CMSid #14
Item #27; CMSid #15
Item #28; CMSid #16
Item #32; CMSid #20
Item #18623; CMSid #19964
Item #33; CMSid #21
Item #34; CMSid #22
Item #36; CMSid #24
Item #37; CMSid #25
Item #35; CMSid #23
Item #38; CMSid #26
Item #39; CMSid #27
Item #40; CMSid #28
Item #41; CMSid #29
Item #18624; CMSid #19965
Item #18625; CMSid #19966
Item #42; CMSid #30
Item #43; CMSid #32
Item #24900; CMSid #30000
Found CMS item with id #30000
Item #45; CMSid #34
Item #46; CMSid #35
Item #47; CMSid #36
Item #48; CMSid #37
Item #50; CMSid #39
Item #49; CMSid #38
Item #51; CMSid #40
Item #52; CMSid #41
Item #53; CMS

Item #333; CMSid #331
Found CMS item with id #331
Item #18687; CMSid #20028
Item #334; CMSid #332
Item #18688; CMSid #20029
Item #323; CMSid #320
Item #18689; CMSid #20030
Item #335; CMSid #333
Item #336; CMSid #334
Item #337; CMSid #335
Item #338; CMSid #336
Item #18690; CMSid #20031
Item #18691; CMSid #20032
Item #18692; CMSid #20033
Item #18693; CMSid #20034
Item #18694; CMSid #20035
Item #340; CMSid #338
Item #339; CMSid #337
Item #341; CMSid #339
Item #18695; CMSid #20036
Item #342; CMSid #340
Item #343; CMSid #341
Item #18696; CMSid #20037
Item #18697; CMSid #20038
Item #344; CMSid #342
Item #345; CMSid #343
Item #346; CMSid #344
Item #347; CMSid #345
Item #348; CMSid #346
Item #349; CMSid #347
Item #351; CMSid #349
Item #350; CMSid #348
Item #18698; CMSid #20039
Item #18034; CMSid #19375
Item #352; CMSid #351
Item #353; CMSid #353
Item #354; CMSid #354
Item #355; CMSid #355
Item #356; CMSid #356
Item #357; CMSid #357
Item #358; CMSid #358
Item #359; CMSid #359
Item #360; CMSid #

Item #593; CMSid #609
Item #588; CMSid #604
Item #589; CMSid #605
Item #591; CMSid #607
Item #590; CMSid #606
Item #594; CMSid #610
Item #595; CMSid #611
Item #596; CMSid #612
Item #597; CMSid #613
Item #598; CMSid #614
Item #599; CMSid #615
Item #600; CMSid #616
Item #601; CMSid #617
Item #602; CMSid #618
Item #603; CMSid #619
Item #604; CMSid #620
Item #605; CMSid #621
Item #606; CMSid #622
Item #607; CMSid #623
Item #608; CMSid #624
Item #609; CMSid #625
Item #18802; CMSid #20143
Item #18803; CMSid #20144
Item #18804; CMSid #20145
Item #610; CMSid #627
Item #611; CMSid #628
Item #612; CMSid #629
Item #613; CMSid #630
Item #614; CMSid #631
Item #18805; CMSid #20146
Item #18806; CMSid #20147
Item #18807; CMSid #20148
Item #18808; CMSid #20149
Item #18809; CMSid #20150
Item #615; CMSid #634
Item #18810; CMSid #20151
Item #616; CMSid #635
Item #618; CMSid #637
Item #619; CMSid #638
Item #620; CMSid #639
Item #18811; CMSid #20152
Item #621; CMSid #640
Item #622; CMSid #642
Item #623; CMS

Item #833; CMSid #877
Item #835; CMSid #879
Item #834; CMSid #878
Item #802; CMSid #836
Item #18932; CMSid #20273
Item #18933; CMSid #20274
Item #840; CMSid #885
Item #839; CMSid #884
Item #18934; CMSid #20275
Item #838; CMSid #883
Item #841; CMSid #886
Item #842; CMSid #887
Item #843; CMSid #888
Item #844; CMSid #889
Item #845; CMSid #890
Item #846; CMSid #891
Item #847; CMSid #892
Item #848; CMSid #893
Item #849; CMSid #894
Item #850; CMSid #895
Item #18064; CMSid #19405
Item #18935; CMSid #20276
Item #851; CMSid #897
Item #864; CMSid #910
Item #856; CMSid #902
Item #852; CMSid #898
Item #858; CMSid #904
Item #853; CMSid #899
Item #854; CMSid #900
Item #857; CMSid #903
Item #855; CMSid #901
Item #859; CMSid #905
Item #862; CMSid #908
Item #860; CMSid #906
Item #861; CMSid #907
Item #863; CMSid #909
Item #865; CMSid #911
Item #866; CMSid #912
Item #867; CMSid #913
Item #868; CMSid #914
Item #18936; CMSid #20277
Item #869; CMSid #915
Item #880; CMSid #926
Item #870; CMSid #916
Item #87

Item #1106; CMSid #1169
Item #18103; CMSid #19444
Item #19013; CMSid #20354
Item #1107; CMSid #1172
Item #1108; CMSid #1173
Item #19014; CMSid #20355
Item #1109; CMSid #1174
Item #1110; CMSid #1175
Item #1111; CMSid #1176
Item #1112; CMSid #1177
Item #1113; CMSid #1178
Item #1114; CMSid #1179
Item #1115; CMSid #1180
Item #1116; CMSid #1181
Item #1117; CMSid #1182
Item #1118; CMSid #1183
Item #1119; CMSid #1184
Item #1120; CMSid #1185
Item #1121; CMSid #1186
Item #24910; CMSid #30010
Found CMS item with id #30010
Item #1122; CMSid #1187
Item #1123; CMSid #1188
Item #1124; CMSid #1189
Item #1125; CMSid #1190
Item #19015; CMSid #20356
Item #1126; CMSid #1191
Item #1127; CMSid #1192
Item #1128; CMSid #1193
Item #1129; CMSid #1194
Item #1130; CMSid #1195
Item #1131; CMSid #1196
Item #1132; CMSid #1197
Item #1133; CMSid #1198
Item #1134; CMSid #1199
Item #1135; CMSid #1200
Item #1136; CMSid #1201
Item #19016; CMSid #20357
Item #19017; CMSid #20358
Item #1137; CMSid #1202
Item #1138; CMSid #1

...

Item #16839; CMSid #18088
Item #17025; CMSid #18301
Item #17026; CMSid #18302
Item #17027; CMSid #18303
Item #17028; CMSid #18304
Item #17029; CMSid #18305
Item #17030; CMSid #18306
Item #17031; CMSid #18307
Item #24528; CMSid #25869
Item #17032; CMSid #18308
Item #17033; CMSid #18309
Item #17034; CMSid #18310
Item #17035; CMSid #18311
Item #17036; CMSid #18312
Item #24529; CMSid #25870
Item #17037; CMSid #18313
Item #17038; CMSid #18314
Item #17039; CMSid #18315
Item #17040; CMSid #18316
Item #24530; CMSid #25871
Item #17041; CMSid #18318
Item #17042; CMSid #18319
Item #17043; CMSid #18320
Item #24531; CMSid #25872
Item #24532; CMSid #25873
Item #24533; CMSid #25874
Item #17044; CMSid #18321
Item #17045; CMSid #18322
Item #17046; CMSid #18323
Item #17047; CMSid #18324
Item #17048; CMSid #18325
Item #17049; CMSid #18326
Item #17050; CMSid #18327
Item #17051; CMSid #18328
Item #17052; CMSid #18329
Item #17059; CMSid #18336
Item #17053; CMSid #18330
Item #17054; CMSid #18331
Item #17055;

Item #17345; CMSid #18631
Item #17325; CMSid #18610
Item #17326; CMSid #18611
Item #17346; CMSid #18632
Item #17347; CMSid #18633
Item #18608; CMSid #19949
Found CMS item with id #19949
Item #17348; CMSid #18634
Item #24591; CMSid #25932
Item #17349; CMSid #18636
Item #17350; CMSid #18637
Item #17351; CMSid #18638
Item #17356; CMSid #18644
Item #17352; CMSid #18639
Item #17353; CMSid #18640
Item #17357; CMSid #18645
Item #17355; CMSid #18642
Item #17358; CMSid #18646
Item #24592; CMSid #25933
Item #17359; CMSid #18647
Item #17360; CMSid #18648
Item #17361; CMSid #18650
Item #17363; CMSid #18652
Item #17362; CMSid #18651
Item #17364; CMSid #18653
Item #24593; CMSid #25934
Item #17365; CMSid #18654
Item #24594; CMSid #25935
Item #24595; CMSid #25936
Item #17367; CMSid #18656
Item #17368; CMSid #18657
Item #17369; CMSid #18658
Item #17366; CMSid #18655
Item #17372; CMSid #18661
Item #24994; CMSid #30089
Found CMS item with id #30089
Item #24597; CMSid #25938
Item #24598; CMSid #25939
Item

Item #17567; CMSid #18863
Item #17568; CMSid #18864
Item #17569; CMSid #18865
Item #18491; CMSid #19832
Item #18492; CMSid #19833
Item #24677; CMSid #26018
Item #17570; CMSid #18868
Item #18540; CMSid #19881
Item #17571; CMSid #18870
Item #17573; CMSid #18873
Item #17575; CMSid #18875
Item #17577; CMSid #18877
Item #17578; CMSid #18878
Item #17579; CMSid #18879
Item #17576; CMSid #18876
Item #24678; CMSid #26019
Item #17580; CMSid #18880
Item #24681; CMSid #26022
Item #24680; CMSid #26021
Item #24682; CMSid #26023
Item #17581; CMSid #18881
Item #24683; CMSid #26024
Item #18239; CMSid #19580
Item #24684; CMSid #26025
Item #24685; CMSid #26026
Item #24686; CMSid #26027
Item #17582; CMSid #18883
Item #17583; CMSid #18884
Item #17584; CMSid #18885
Item #17585; CMSid #18886
Item #17587; CMSid #18888
Found CMS item with id #18888
Item #17588; CMSid #18889
Item #17589; CMSid #18890
Item #17590; CMSid #18891
Item #24687; CMSid #26028
Item #24688; CMSid #26029
Item #17591; CMSid #18892
Item #17

Item #24797; CMSid #26138
Item #17763; CMSid #19076
Item #17764; CMSid #19077
Item #24798; CMSid #26139
Item #17754; CMSid #19065
Item #24799; CMSid #26140
Item #17765; CMSid #19078
Item #17766; CMSid #19079
Item #17767; CMSid #19080
Item #17755; CMSid #19066
Item #24800; CMSid #26141
Item #17768; CMSid #19081
Item #17769; CMSid #19082
Item #17770; CMSid #19083
Item #17771; CMSid #19084
Item #17772; CMSid #19085
Item #17773; CMSid #19086
Item #17774; CMSid #19087
Item #24801; CMSid #26142
Item #17775; CMSid #19088
Item #17776; CMSid #19089
Item #24802; CMSid #26143
Item #17778; CMSid #19091
Item #17779; CMSid #19092
Item #24803; CMSid #26144
Item #17780; CMSid #19093
Item #17781; CMSid #19094
Item #17782; CMSid #19095
Item #17783; CMSid #19096
Item #17784; CMSid #19097
Item #17785; CMSid #19098
Item #17786; CMSid #19099
Item #17787; CMSid #19100
Item #24804; CMSid #26145
Item #17788; CMSid #19101
Item #17789; CMSid #19102
Item #17790; CMSid #19103
Item #24805; CMSid #26146
Item #24806;

Item #17990; CMSid #19330
Item #17986; CMSid #19326
Item #17991; CMSid #19331
Item #17992; CMSid #19332
Item #17777; CMSid #19090
Item #17993; CMSid #19333
Item #17994; CMSid #19334
Item #17995; CMSid #19335
Item #17996; CMSid #19336
Item #17997; CMSid #19337
Item #24884; CMSid #26225
Item #17998; CMSid #19338
Item #17999; CMSid #19339
Item #24885; CMSid #26226
Item #18000; CMSid #19340
Item #24883; CMSid #26224
Item #18001; CMSid #19341
Item #24886; CMSid #26227
Item #18002; CMSid #19342
Item #18003; CMSid #19343
Item #18004; CMSid #19344
Item #24887; CMSid #26228
Item #24888; CMSid #26229
Item #24890; CMSid #26231
Item #24889; CMSid #26230
Item #24891; CMSid #26232
Item #24892; CMSid #26233
Item #24893; CMSid #26234
Item #24894; CMSid #26235
Item #18005; CMSid #19345
Item #24895; CMSid #26236
Item #24896; CMSid #26237
Item #24897; CMSid #26238
Item #18006; CMSid #19346
Item #18007; CMSid #19347
Item #18008; CMSid #19348
Item #18009; CMSid #19349
Item #18010; CMSid #19350
Item #18011;

In [14]:
items_to_keep = used_items.union(matching_cmsitems)
print(len(items_to_keep))

711


In [19]:
count = 0
deleted = 0
for item in Item.objects.all():
    count += 1
    if item.id not in items_to_keep:
        print(f"Deleting #{item.id:<6}|{item.name}")
        item.delete()
        deleted += 1
print(f"Processed: {count}, deleted: {deleted}, kept: {count-deleted}")

Deleting #19    |0.15% POTASSIUM CHLORIDE AND 0.9% SODIUM CHLORIDE IV INF
Deleting #20    |0.15% POTASSIUM CHLORIDE AND 5% GLUCOSE IV INF
Deleting #21    |0.15% POTASSIUM CHLORIDE IN 2.5% GLUCOSE AND 0.45% SODIUM CHLORIDE IV INF
Deleting #16    |0.15% POT CHL/0.9% SOD CHL IV INF
Deleting #17    |0.15% POT CHL/5% GLUC IV INFUSION
Deleting #18    |0.15% POT CL/2.5% GLUC/0.45% SOD CL INJ
Deleting #22    |0.18% SOD CHLORIDE & 4.3% GLUCOSE INJ
Deleting #23    |0.18% SODIUM CHLORIDE 4.3% DEXTROSE INJ
Deleting #24    |0.18% SODIUM CHLORIDE AND 4.3% DEXTROSE INJ
Deleting #25    |0.18% SODIUM CHLORIDE AND 4.3% GLUCOSE INTRAVENOUS INFUSION
Deleting #14    |0.1% CHLORHEXIDINE & 1% CETRIMIDE SOLN
Deleting #29    |0.3% POTASSIUM CHLORIDE AND 0.9% SODIUM CHLORIDE IV INF
Deleting #30    |0.3% POTASSIUM CHLORIDE AND 5% GLUCOSE IV INF
Deleting #31    |0.3% POTASSIUM CHLORIDE IN 2.5% GLUCOSE AND 0.45% SODIUM CHLORIDE IV INF
Deleting #26    |0.3% POT CHL/0.9% SOD CHL IV INF
Deleting #27    |0.3% POT CHL/

Deleting #218   |ACNOLAIC GEL 20% W/W
Deleting #219   |ACNOTIN 10 CAP 10MG
Deleting #220   |ACNOTIN 20 CAP 20MG
Deleting #18659 |ACNY-10 CREAM 10%
Deleting #221   |ACODIN COUGH SYRUP
Deleting #222   |ACOFED COUGH SYRUP
Deleting #18660 |ACOLON ORABASE
Deleting #223   |ACONIL CAP 250MG
Deleting #224   |ACONIL CAP R/Y 250MG
Deleting #225   |ACONIL FORT CAP 500MG
Deleting #226   |ACONIL FORTE CAP G/G 500MG
Deleting #227   |ACONIL SPECIAL CAP 500MG
Deleting #228   |ACOSOLVAN SYRUP 15MG/5ML
Deleting #229   |ACOSOLVAN SYRUP 30MG/5ML
Deleting #230   |ACOSOLVAN TAB 30MG
Deleting #231   |ACRIFLAVINE SOLN 0.1% (QUALITY)
Deleting #232   |ACRINOL SOLUTION 0.1%
Deleting #233   |ACRINOL SOLUTION 0.1% (JEN SHENG)
Deleting #234   |ACROWN CAP
Deleting #235   |ACTA CREAM 0.05%
Deleting #236   |ACTA CREAM 0.1%
Deleting #237   |ACTAL PLUS CHEWABLE TAB
Deleting #238   |ACTAL TAB
Deleting #239   |ACTAPIN TAB 10MG
Deleting #240   |ACTAPIN TAB 5MG
Deleting #241   |ACTEIN EFFERVESCENT TAB 600MG
Deleting #18661 

Deleting #381   |ADVANCE PARAMOL EXTRA TAB
Deleting #18706 |ADVANCE PURE CARE VITAMIN C 1000MG PLUS ZINC 10MG EFFERVESCENT TABLET
Deleting #18707 |ADVANCE RASH CREAM FOR CHILDREN
Deleting #384   |ADVANTAGE 100 SOLN 100MG/ML (VET)
Deleting #385   |ADVANTAGE 250 SOLN 250MG/2.5ML (VET)
Deleting #388   |ADVANTAGE 400 SOLN 400MG/4ML (VET)
Deleting #386   |ADVANTAGE 40 FOR CATS SOLUTION 40MG/0.4ML (VET)
Deleting #387   |ADVANTAGE 40 SOLN 40MG/0.4ML (VET)
Deleting #389   |ADVANTAGE 80 FOR CATS SOLUTION 80MG/0.8ML (VET)
Deleting #18407 |ADVANTAN CREAM 0.1%
Deleting #18408 |ADVANTAN OINT 0.1%
Deleting #383   |ADVANT TAB 16MG
Deleting #18708 |ADVANT TAB 8MG
Deleting #390   |ADVATE FOR INJ 1000IU
Deleting #391   |ADVATE FOR INJ 250IU
Deleting #392   |ADVATE INJ 1500IU
Deleting #393   |ADVATE INJ 500IU
Deleting #394   |ADVATUSIN COUGH SUSPENSION
Deleting #18709 |ADVIL LIQUID CAPSULES 200MG
Deleting #395   |ADVIL TAB 200MG
Deleting #396   |ADVIR CREAM 5%
Deleting #18710 |AERIUS D-12 MODIFIED RELEAS

Deleting #18770 |ALLEGRO NASAL SPRAY 0.05%
Deleting #18771 |ALLENOR TABLET 0.75MG
Deleting #521   |ALLEN'S COUGH SYRUP
Deleting #522   |ALLEN'S COUGH SYRUP - APRICOT FLAVOUR
Deleting #18772 |ALLEN'S COUGH SYRUP - CM FORMULA
Deleting #18773 |ALLENTOP TAB 70MG
Deleting #524   |ALLERCIN TAB 5MG
Deleting #18775 |ALLERCLEAR TABLETS 5MG
Deleting #526   |ALLERDINE TAB 10MG
Deleting #527   |ALLEREX CAP
Deleting #528   |ALLEREX EYE DROPS
Deleting #529   |ALLERFAST-120 TAB 120MG
Deleting #18776 |ALLERFAST-180 TAB 180MG
Deleting #530   |ALLERGIN CREAM
Deleting #18777 |ALLERGO-COMOD EYE DROPS 2%W/V
Deleting #531   |ALLERGOCROM EYE DROPS 2%
Deleting #532   |ALLERGONE CAP 10MG
Deleting #18778 |ALLERGOPOS EYE DROPS
Deleting #533   |ALLERGOTIN OPHTH SOLN 2%
Deleting #534   |ALLERGY EXTENDED RELEASE CAP 12MG
Deleting #18779 |ALLERIEF ORAL SOLUTION 2MG/5ML
Deleting #535   |ALLERMIN EXPECTORANT
Deleting #536   |ALLERMIN INJ 5MG/ML
Deleting #537   |ALLERMIN SYRUP 0.05%
Deleting #538   |ALLERMIN SYRUP 5MG/

Deleting #676   |AMERICAN HEALTH ABC PLUS WITH LUTEIN TAB
Deleting #677   |AMERICAN HEALTH C-1000MG WITH ROSE HIPS TAB
Deleting #678   |AMERICAN HEALTH C-500MG WITH ROSE HIPS TAB
Deleting #679   |AMERICAN HEALTH CORAL CALCIUM CAP 185MG
Deleting #675   |AMERICAN HEALTH  CORAL CALCIUM COMPLEX 500MG W/D, MAGNESIUM CAP
Deleting #680   |AMERICAN HEALTH D/S GLUCOSAMINE CHONDROITIN TAB
Deleting #681   |AMERICAN HEALTH MAXIMUM STRENGTH GLUCOSAMINE TAB 1500MG
Deleting #682   |AMERICAN HEALTH NATURAL E-200IU CAP 200IU
Deleting #683   |AMERICAN HEALTH NATURAL VITAMIN A & D CAP
Deleting #684   |AMERICAN HEALTH SUPER B COMPLEX WITH VITAMIN C TAB
Deleting #685   |AMERICAN HEALTH VITAMIN B2 TAB 100MG
Deleting #687   |AMERICAN HEALTH VITAMIN E-1000IU CAP 1000IU
Deleting #686   |AMERICAN HEALTH VITAMIN E CAP 400IU
Deleting #688   |AMERICAN NUTRITION CO. VITAMIN C WITH ROSEHIPS TAB 500MG
Deleting #689   |AMERICOLD TAB
Deleting #18840 |AMERIDE TAB 2MG
Deleting #690   |AMERIDEX COD LIVER OIL CAP
Deleting 

...

Deleting #17422 |VITAMIN E CAP 400IU (PHARMETICS)
Deleting #17405 |VITAMIN E CAP 400 IU (PURITAN'S PRIDE)
Deleting #17401 |VITAMIN E CAP 400 IU 'ROYALE'
Deleting #17423 |VITAMIN E CAP 400IU (R P SCHERER)
Deleting #17426 |VITAMIN E CAP 400IU VITA-LIFE
Deleting #17424 |VITAMIN E CAP 400IU (VITAMATE)
Deleting #17413 |VITAMIN E CAP 400IU 'WILPHARM'
Deleting #17427 |VITAMIN E CAP 800IU (JAMIESON)
Deleting #24607 |VITAMINEN ACTIVE TAB
Deleting #17428 |VITAMIN E SOFTGEL 400IU (SWISS CAPS)
Deleting #24603 |VITAMIN E SOFTGEL CAP 200IU
Deleting #24604 |VITAMIN E SOFTGEL CAPSULE 400IU
Deleting #24605 |VITAMIN E SOFTGELS CAPSULES 400 IU
Deleting #17429 |VITAMIN E TAB 50IU S.C. (JEN SHENG)
Deleting #17430 |VITAMIN E TAB 50MG
Deleting #17373 |VITAMIN E (WATER SOLUBLE) CAP 400 IU (PURITAN'S PRIDE)
Deleting #17431 |VITAMIN K 1 (DSM)
Deleting #17432 |VITAMIN K1 INJ 10MG/ML
Deleting #17433 |VITAMIN K1 INJ 1MG/0.5ML
Deleting #24606 |VITAMIN K1 INJECTION 10MG/ML
Deleting #17434 |VITAMIN MINERAL GERI CAP 1

Deleting #24662 |VOTRIENT TAB 400MG (SPAIN)
Deleting #16679 |V-ROHTO EYE DROPS
Deleting #17534 |VS SASSOON ANTI-DANDRUFF SHAMPOO 1%-CHINA
Deleting #16680 |V-THIAMIN TAB 100MG
Deleting #24663 |VULTIN 100 CAP 100MG
Deleting #24664 |VULTIN 300 CAP 300MG
Deleting #17535 |VUMON INJ FOR IV INFUSION 50MG/5ML
Deleting #17536 |VURDON GEL 1%
Deleting #24665 |VYIEW TOP EYE DROPS 0.5%
Deleting #24666 |VYIEW TOP EYE OINTMENT 1%
Deleting #17537 |VYTORIN TAB 10MG/40MG
Deleting #17538 |VYTORIN TAB 10MG/80MG
Deleting #17546 |WADA KIDS CALCIUM TAB (CHEWABLE)
Deleting #17547 |WAGON CREAM
Deleting #17548 |WAH KIN GOEASY ENTERIC COATED LAXATIVE TAB 5MG
Deleting #17549 |WAH KIN XUANLEJING CREAM 1%
Deleting #24668 |WAH TAT PATCH 30MG
Deleting #24669 |WAHTATSY COUGH SYRUP
Deleting #17550 |WAH TAT TAB 150MG
Deleting #24670 |WAIFULSIN VITAMIN B1-B6-B12 TABLETS
Deleting #17551 |WAIL TREATMENT OF HEAD LICE LIQUID 25%
Deleting #17552 |WAIMIN COLD CAP
Deleting #17553 |WAIMIN FORTE CAP
Deleting #24671 |WAIYSIN VITAM

Deleting #17675 |WINRHO SDF FOR INJ 600IU
Deleting #24727 |WINRHO SDF LIQUID FOR INJECTION 1500IU
Deleting #24728 |WINRHO SDF LIQUID FOR INJECTION 600IU
Deleting #24729 |WINSEDYL COUGH SYRUP
Deleting #17676 |WINSLET CREAM
Deleting #17677 |WINSODERMA CREAM
Deleting #17678 |WINSOLVE-B OINTMENT
Deleting #17679 |WINTAMOL TAB 300MG
Deleting #24725 |WIN WIN COUGH SYRUP
Deleting #17680 |WISEMAN BETA CAROTENE CAP 10000IU
Deleting #17681 |WISEMAN CALCIUM + VITAMIN D TAB
Deleting #17682 |WISEMAN DAILY-E 1000 CAP 1000IU
Deleting #17683 |WISEMAN DAILY-E 400 CAP 400IU
Deleting #17684 |WISEMAN POLENTAR COD LIVER OIL CAP
Deleting #24730 |WISEMAN VITAMIN E CAPSULES 1000IU
Deleting #24731 |WISEMAN VITAMIN E CAPSULES 400IU
Deleting #17685 |WITAPOLIN SYRUP
Deleting #24732 |WO CHONG ENEMA
Deleting #17686 |WOHON CORN PLASTERS 32MG/STRIP
Deleting #17687 |WONCOX CAP 7.5MG
Deleting #17688 |WONFLOXIN TAB 100MG
Deleting #24733 |WONTAC TAB 150MG
Deleting #17689 |WONTIZEM RETARD TAB 90MG
Deleting #24734 |WONTRAN 

Deleting #17806 |ZAROM 250 CAP 250MG
Deleting #17807 |ZAROXOLYN TAB 2.5MG
Deleting #17808 |ZAROXOLYN TAB 5MG
Deleting #24816 |ZARZIO SOLUTION FOR INJECTION OR INFUSION IN PRE-FILLED SYRINGE 30MU/0.5ML
Deleting #24817 |ZATAMIN TABLETS 10MG
Deleting #17809 |ZATHRIN 250 TAB 250MG
Deleting #17810 |ZATHRIN 500 TAB 500MG
Deleting #17811 |ZATIX COUGH SYRUP
Deleting #17812 |ZAVEDOS CAP 5MG
Deleting #17813 |ZAVEDOS FOR INJ IV
Deleting #24818 |ZEAGRA 100 TAB 100MG
Deleting #24819 |ZECID-20 CAP 20MG (ENTERIC-COATED GRANULES)
Deleting #17814 |ZECO EXPECTORANT
Deleting #24820 |ZECON 150 CAP 150MG
Deleting #17815 |ZECON CREAM
Deleting #17816 |ZEEFRA GE CAP 500MG
Deleting #17817 |ZEETAMIN-500 TAB 500MG
Deleting #18361 |ZEFFIX TAB 100MG
Deleting #17818 |ZEGAVIT CAPLET
Deleting #24821 |ZELAVEL TABLETS 200MG
Deleting #24822 |ZELBORAF FILM-COATED TAB 240MG
Deleting #17819 |ZELDOX CAP 20MG
Deleting #17820 |ZELDOX CAP 40MG
Deleting #17821 |ZELDOX CAP 60MG
Deleting #17822 |ZELDOX CAP 80MG
Deleting #17823 |Z

Deleting #17954 |ZORAMIX-F CAP 500MG
Deleting #17955 |ZORAX CREAM 5%
Deleting #17956 |ZORAXIN SUSPENSION 200MG/5ML
Deleting #17957 |ZORAXIN TAB 400 MG
Deleting #17959 |ZORYL-1 TAB 1 MG
Deleting #17958 |ZORYL - 2 TAB 2MG
Deleting #17960 |ZOSPAR-200 TAB 200MG
Deleting #17961 |ZOSTATIN F.C. TAB 20MG "S.C."
Deleting #17965 |ZOSTIN-FORTE TAB 400MG
Deleting #17963 |ZOSTIN OINT 5%
Deleting #17966 |ZOSTINOL OINTMENT 5%
Deleting #17967 |ZOSTINOL TAB 200MG
Deleting #17964 |ZOSTIN TAB 200MG
Deleting #17968 |ZOTAX-250 TAB 250MG
Deleting #17969 |ZOTEX TAB 0.5MG
Deleting #17971 |ZOTOCORT CR
Deleting #17970 |ZOTO MOUTHWASH 0.2%
Deleting #17973 |ZOVATIN TAB 10MG
Deleting #17972 |ZOVATIN TAB 20MG
Deleting #17974 |ZOVATIN TAB 40MG
Deleting #17975 |ZOVAX OINTMENT 5%
Deleting #17976 |ZOVAX TAB 200MG
Deleting #17977 |ZOVICORT GEL 0.1%
Deleting #17978 |ZOVIRAX COLD SORE CREAM 5%
Deleting #17979 |ZOVIRAX CREAM 5%
Deleting #17980 |ZOVIRAX IV FOR INTRAVENOUS INFUSION250MG
Deleting #17981 |ZOVIRAX OPHTH OINT 3%

In [4]:
for drug in RegisteredDrug.objects.all():
    if not drug.itemid:
        try:
            item_obj = Item.objects.get(reg_no=drug.reg_no)
        except Item.DoesNotExist:
            item_obj = None
        if item_obj:
            # Update RegisteredDrug with matching item_obj
            drug.itemid = item_obj.id
            print(f"Updated {drug.reg_no}-Item #{drug.itemid}|{drug.name}")
            drug.save()
    
    

Updated HK01640-Item #25061|PARACETAMOL TAB 500MG
Updated HK01666-Item #25046|DIPROPHYLLINE TAB 200MG
Updated HK05397-Item #25060|CHLORPHENIRAMINE TAB 4MG
Updated HK05406-Item #25051|VITAMIN B12 INJ 1000MCG/ML
Updated HK08072-Item #25047|HYDROCORTISONE CREAM 1%
Updated HK14637-Item #25043|AMOXYCILLIN CAP 250MG
Updated HK16771-Item #25054|CHLORPHENIRAMINE MALEATE INJ 10MG/ML
Updated HK22740-Item #25045|DIAZEPAM TAB 1MG
Updated HK30103-Item #25048|MILZINE TAB
Updated HK37156-Item #25058|SCOPOLAMINE METHYLBROMIDE TAB 1MG
Updated HK40509-Item #25044|COCILLANA SYRUP
Updated HK46484-Item #25050|TAMIFLU CAP 75MG
Updated HK48244-Item #25057|PROPIRIN ENTERIC COATED TAB 100MG
Updated HK50090-Item #25049|NAPROXEN TAB 250MG
Updated HK58642-Item #25059|TIMABAK EYE DROPS 0.5%
Updated HK59516-Item #25052|AMBROXOL TAB 30MG
Updated HK60652-Item #25053|BROMHEXINE SYRUP 4MG/5ML
Updated HK60796-Item #25056|LEVOTHYROXINE TAB 100MCG
Updated HK61674-Item #25055|DONEPEZIL HYDROCHLORIDE TABLETS 5MG
Updated HK6